In [ ]:
!pip install pytorch-pretrained-bert pytorch-nlp
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import torch

In [ ]:
#Check if GPU is present for processing
if torch.cuda.is_available():
  device=torch.device("cuda")
else:
  device="cpu"
print(device)


cuda


In [ ]:
#Load data
df=pd.read_csv('/content/drive/MyDrive/Data/ToxicTextClassification/train.csv')
df=df[['comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]
df=df[:6000]
df.head(3)

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [ ]:
sentences=df['comment_text'].values
# labels=df[['toxic']].values
labels=df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']].values
labels[:5]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
input_ids=[]
MAXLEN=0
for sen in sentences:
  encoded=tokenizer.encode(sen,add_special_tokens=True,truncation=True)
  MAXLEN=max(MAXLEN,len(encoded))
  input_ids.append(encoded)
input_ids[:1],MAXLEN

([[101,
   7526,
   2339,
   1996,
   10086,
   2015,
   2081,
   2104,
   2026,
   5310,
   18442,
   13076,
   12392,
   2050,
   5470,
   2020,
   16407,
   1029,
   2027,
   4694,
   1005,
   1056,
   3158,
   9305,
   22556,
   1010,
   2074,
   8503,
   2006,
   2070,
   3806,
   2044,
   1045,
   5444,
   2012,
   2047,
   2259,
   14421,
   6904,
   2278,
   1012,
   1998,
   3531,
   2123,
   1005,
   1056,
   6366,
   1996,
   23561,
   2013,
   1996,
   2831,
   3931,
   2144,
   1045,
   1005,
   1049,
   3394,
   2085,
   1012,
   6486,
   1012,
   16327,
   1012,
   4229,
   1012,
   2676,
   102]],
 512)

In [ ]:
from keras_preprocessing.sequence import pad_sequences

MAX_LEN=64

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post",padding="post",
                          value=tokenizer.pad_token_id)



Padding token: "[PAD]", ID: 0


In [ ]:
#Adding attention masks to differentiate between padding tokens and sentence tokens. 
attention_mask=[]
for inputs in input_ids:
  attention=[int(i>0) for i in inputs]
  attention_mask.append(attention)
# attention_mask[:5]

In [ ]:
#Training and validation set

train_len=int(0.9*len(input_ids))
print("training length:",train_len)

train_inputs=torch.tensor(input_ids[:train_len])
train_masks=torch.tensor(attention_mask[:train_len])
train_label=torch.tensor(labels[:train_len])
val_inputs=torch.tensor(input_ids[train_len:])
val_masks=torch.tensor(attention_mask[train_len:])
val_label=torch.tensor(labels[train_len:])



training length: 5400


In [ ]:
#creaate tensor datasets 
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
validation_data = TensorDataset(val_inputs, val_masks, val_label)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 6, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
#Adam optimizer with weigth decay fixes 
import torch.optim as optim
# from warmup_scheduler_pytorch import WarmUpScheduler
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)
# scheduler_warmup = WarmUpScheduler(optimizer, multiplier=1, total_epoch=5, after_scheduler=scheduler)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 8

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler_warmup = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 5, # Default value in run_glue.py
                                            num_training_steps = total_steps)
scheduler

In [ ]:
import torch.nn as nn
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup, BertPreTrainedModel
bert_model_name = 'bert-base-uncased'
class BertClassifier(nn.Module):
    
    def __init__(self, bert: BertModel, num_classes: int):
        super().__init__()
        self.bert = bert
        self.classifier = nn.Linear(bert.config.hidden_size, num_classes)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None,
                
            labels=None):
        outputs = self.bert(input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               position_ids=position_ids,
                               head_mask=head_mask)
        cls_output = outputs[1] # batch, hidden
        cls_output = self.classifier(cls_output) # batch, 6
        cls_output = torch.sigmoid(cls_output)
        # cls_output=cls_output.detach()
        # labels=labels.numpy()
        criterion = nn.BCELoss()
        loss = 0
        if labels is not None:
            loss = criterion(cls_output, labels.float())
        return loss, cls_output

    def predict(self,comment):
      outputs = self.bert(comment)
      cls_output = outputs[1] # batch, hidden
      cls_output = self.classifier(cls_output) # batch, 6
      # cls_output = torch.sigmoid(cls_output)
      print("Predicted:",cls_output)


model = BertClassifier(BertModel.from_pretrained(bert_model_name), 6).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#Model training phase 
import random
import numpy as np
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

model.train()
for e in range(epochs):
  scheduler_warmup.step(e)
  
  total_loss=0
  for step,batch in enumerate(train_dataloader):
    input,mask,label=batch
    input,mask,label=input.to(device),mask.to(device),label.to(device)

    loss=model(input,attention_mask=mask,labels=label,token_type_ids=None)

    total_loss += loss[0].item()
    
    optimizer.zero_grad()
    loss[0].backward()
    optimizer.step()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

  avg_train_loss = total_loss / len(train_dataloader)            
  scheduler.step(avg_train_loss)  
    # Store the loss value for plotting the learning curve.
  loss_values.append(avg_train_loss)

  print("  Average training loss: {0:.2f}".format(avg_train_loss))




/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:163: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.

  Average training loss: 0.70
  Average training loss: 0.70
  Average training loss: 0.70
  Average training loss: 0.70
  Average training loss: 0.70
  Average training loss: 0.70
  Average training loss: 0.70
  Average training loss: 0.70


In [ ]:
# df=pd.read_csv('/content/drive/MyDrive/Data/ToxicTextClassification/train.csv')

comment=df.iloc[2603]['comment_text']
comment

'I would also like to say \n\nthat not only are you a masturbate, but you are also gay lol'

In [ ]:
encoded=tokenizer.encode(comment,add_special_tokens=True,truncation=True)
print("encoded: ",encoded)
comment_pad = pad_sequences([encoded], maxlen=MAX_LEN, dtype="long", truncating="post",padding="post",
                          value=tokenizer.pad_token_id)
print("comment_pad: ",comment_pad)
model.eval()
comment_pad=torch.tensor(comment_pad)
comment_pad=comment_pad.to(device)
model.predict(comment_pad)

encoded:  [101, 1045, 2052, 2036, 2066, 2000, 2360, 2008, 2025, 2069, 2024, 2017, 1037, 15429, 3126, 20179, 1010, 2021, 2017, 2024, 2036, 5637, 8840, 2140, 102]
comment_pad:  [[  101  1045  2052  2036  2066  2000  2360  2008  2025  2069  2024  2017
   1037 15429  3126 20179  1010  2021  2017  2024  2036  5637  8840  2140
    102     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0]]
Predicted: tensor([[-0.2451,  0.0400, -0.2079,  0.4235, -0.3027,  0.2502]],
       device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
df.iloc[2603][['comment_text','toxic','severe_toxic','obscene','threat','insult','identity_hate']]

comment_text     I would also like to say \n\nthat not only are...
toxic                                                            1
severe_toxic                                                     0
obscene                                                          0
threat                                                           0
insult                                                           1
identity_hate                                                    0
Name: 2603, dtype: object